In [41]:
import pandas as pd
import requests

In [99]:
# Define year range for data ingestion
start_year = 2015
end_year = 2024

years_list = [year for year in range(start_year, end_year+1)]

In [101]:
# load fish count data

fish_years = ",".join(map(str, sorted(years_list, reverse=True)))

fish_url = "https://www.adfg.alaska.gov/sf/FishCounts/index.cfm?ADFG=export.JSON&countLocationID=40&year=" + fish_years + "&speciesID=420"

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
}

response = requests.get(fish_url, headers=headers)

if response.status_code == 200:
    fish_json = response.json()
else:
    print(f"Failed to fetch data: {response.status_code}, {response.text}")

fish_data = pd.DataFrame(fish_json['DATA'], columns=fish_json['COLUMNS'])
fish_data

,YEAR,COUNTDATE,FISHCOUNT,SPECIESID,COUNTLOCATIONID,COUNTLOCATION,SPECIES
0,2024,"July, 01 2024 00:00:00",6366,420,40,Kenai River (late-run sockeye),Sockeye
1,2024,"July, 02 2024 00:00:00",5484,420,40,Kenai River (late-run sockeye),Sockeye
2,2024,"July, 03 2024 00:00:00",9630,420,40,Kenai River (late-run sockeye),Sockeye
3,2024,"July, 04 2024 00:00:00",13590,420,40,Kenai River (late-run sockeye),Sockeye
4,2024,"July, 05 2024 00:00:00",18120,420,40,Kenai River (late-run sockeye),Sockeye
...,...,...,...,...,...,...,...
523,2015,"August, 22 2015 00:00:00",19153,420,40,Kenai River (late-run sockeye),Sockeye
524,2015,"August, 23 2015 00:00:00",24346,420,40,Kenai River (late-run sockeye),Sockeye
525,2015,"August, 24 2015 00:00:00",26910,420,40,Kenai River (late-run sockeye),Sockeye
526,2015,"August, 25 2015 00:00:00",23109,420,40,Kenai River (late-run sockeye),Sockeye
